### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.

In [23]:
from platform import python_version
import sys
f'Python version: {python_version()}'
f'Python executable:{sys.executable}'
f'*********** Python version: details ************'
print(sys.version)
print(sys.version_info)

#get_ipython().run_line_magic('config', 'IPCompleter.greedy=True')
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"

'Python version: 3.12.0'

'Python executable:c:\\Users\\u3184\\azdevops\\pevosdcgh\\AI-Gateway\\.venv\\Scripts\\python.exe'

'*********** Python version: details ************'

3.12.0 (tags/v3.12.0:0fb18b0, Oct  2 2023, 13:03:39) [MSC v.1935 64 bit (AMD64)]
sys.version_info(major=3, minor=12, micro=0, releaselevel='final', serial=0)


In [27]:
import os
import json
import datetime
import platform
import subprocess

oss=platform.system()
lab_prefix="ail-pevo1-"
tool_prefix="st-pevo1-"
deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"{lab_prefix}{deployment_name}" # change the name to match your naming style

def log(stdout, name, action):
    if stdout.n.startswith("ERROR"):
        print("👎🏻 ", name, " was NOT ", action, ": ", stdout)
    else:
        print("👍🏻 ", name, " was ", action, " ⌚ ", datetime.datetime.now().time())

#deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} -o json

cmdStr= f"az deployment group show --name {deployment_name} -g {resource_group_name} -o json"
cmd=cmdStr.split(' ')
result = subprocess.run(cmd, capture_output=True, text=True, shell=True)

cmd_rtc = result.returncode
cmd_out = result.stdout
cmd_err = result.stderr

#if ( (oss=='Windows' and exit_code != 1) or (oss=='Linux' and exit_code != 0) ): 
if ( cmd_rtc != 0 ): 
        print("Deployment Error", cmd_rtc, deployment_name, resource_group_name)
        print(cmd_err)
        raise SystemExit("Stopping the execution of the cell.") 

deployment = json.loads(deployment_stdout.n)
for resource in deployment.get("properties").get("outputResources"):
    resource_id = resource.get("id")
    try:
        query = "\"{type:type, name:name, location:location}\""
        resource_stdout = ! az resource show --id {resource_id} --query {query} -o json
        resource = json.loads(resource_stdout.n)
        resource_name = resource.get("name")
        resource_type = resource.get("type")  
        if resource_type == "Microsoft.CognitiveServices/accounts":
            resource_location = "\"" + resource.get("location") + "\""
            delete_stdout = ! az cognitiveservices account delete -g {resource_group_name} -n {resource_name}
            log(delete_stdout, resource_name, "deleted")
            delete_stdout = ! az cognitiveservices account purge -g {resource_group_name} -n {resource_name} -l {resource_location}
            log(delete_stdout, resource_name, "purged")
        elif resource_type == "Microsoft.ApiManagement/service":
            resource_location = "\"" + resource.get("location") + "\""
            delete_stdout = ! az apim delete -n {resource_name} -g {resource_group_name} -y
            log(delete_stdout, resource_name, "deleted")
            delete_stdout = ! az apim deletedservice purge --service-name {resource_name} --location {resource_location}
            log(delete_stdout, resource_name, "purged")
    except:
        print("✌🏻 ", resource_id, " ignored")
delete_stdout = ! az group delete --name {resource_group_name} -y
log(delete_stdout, resource_group_name, "deleted")



['az', 'deployment', 'group', 'show', '--name', 'built-in-logging', '-g', 'ail-pevo1-built-in-logging', '-o', 'json']
Exit Code: 3
Output:

Error:
ERROR: (ResourceGroupNotFound) Resource group 'ail-pevo1-built-in-logging' could not be found.
Code: ResourceGroupNotFound
Message: Resource group 'ail-pevo1-built-in-logging' could not be found.



SystemExit: Stopping the execution of the cell.

c:\Users\u3184\azdevops\pevosdcgh\AI-Gateway\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
